In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{IntegerType, StringType, StructField, StructType,MapType}
import org.apache.spark.sql.{DataFrame}
import org.apache.spark.sql.functions
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.{Pipeline,PipelineModel}
import org.apache.spark.ml.feature.Imputer

Intitializing Scala interpreter ...

Spark Web UI available at http://26.26.26.1:4040
SparkContext available as 'sc' (version = 2.4.8, master = local[*], app id = local-1625981282042)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{IntegerType, StringType, StructField, StructType, MapType}
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.feature.Imputer


In [108]:
import java.time.LocalDate
import scala.collection.JavaConverters;

import java.time.LocalDate
import scala.collection.JavaConverters


In [2]:
val spark = SparkSession
    .builder()
    .appName("Utils")
    .master("local[2]")
    .getOrCreate()

val sc = spark.sparkContext

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@1819558f
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@69f3aa98


In [4]:
val data = Seq(("Banana",1000,"2021-07-01"), ("Carrots",1500,"2021-07-01"), ("Beans",1600,"2021-07-01"),
                ("Orange",2000,"2021-07-01"),("Orange",2000,"2021-07-01"),("Banana",400,"2021-07-02"),
                ("Carrots",1200,"2021-07-02"),("Beans",1500,"2021-07-02"),("Orange",4000,"2021-07-02"),
                ("Banana",2000,"2021-07-03"),("Carrots",2000,"2021-07-03"),("Beans",2000,"2021-07-04"))

import spark.sqlContext.implicits._
val df = data.toDF("product","amount","dt")
df.show()

+-------+------+----------+
|product|amount|        dt|
+-------+------+----------+
| Banana|  1000|2021-07-01|
|Carrots|  1500|2021-07-01|
|  Beans|  1600|2021-07-01|
| Orange|  2000|2021-07-01|
| Orange|  2000|2021-07-01|
| Banana|   400|2021-07-02|
|Carrots|  1200|2021-07-02|
|  Beans|  1500|2021-07-02|
| Orange|  4000|2021-07-02|
| Banana|  2000|2021-07-03|
|Carrots|  2000|2021-07-03|
|  Beans|  2000|2021-07-04|
+-------+------+----------+



data: Seq[(String, Int, String)] = List((Banana,1000,2021-07-01), (Carrots,1500,2021-07-01), (Beans,1600,2021-07-01), (Orange,2000,2021-07-01), (Orange,2000,2021-07-01), (Banana,400,2021-07-02), (Carrots,1200,2021-07-02), (Beans,1500,2021-07-02), (Orange,4000,2021-07-02), (Banana,2000,2021-07-03), (Carrots,2000,2021-07-03), (Beans,2000,2021-07-04))
import spark.sqlContext.implicits._
df: org.apache.spark.sql.DataFrame = [product: string, amount: int ... 1 more field]


In [7]:
df.printSchema()

root
 |-- product: string (nullable = true)
 |-- amount: integer (nullable = false)
 |-- dt: string (nullable = true)



In [139]:
def sortByDate(s1: String, s2: String) = {
    val d1 = LocalDate.parse(s1)
    val d2 = LocalDate.parse(s2)
    d2.isAfter(d1)
}

sortByDate: (s1: String, s2: String)Boolean


In [140]:
val dts = df.select($"dt").distinct.collect.toSeq.map(x=>x(0)).asInstanceOf[Seq[String]].sortWith(sortByDate)

val pivotDf = df.groupBy("product").pivot("dt",dts).sum("amount")
pivotDf.show()

+-------+----------+----------+----------+----------+
|product|2021-07-01|2021-07-02|2021-07-03|2021-07-04|
+-------+----------+----------+----------+----------+
| Orange|      4000|      4000|      null|      null|
|  Beans|      1600|      1500|      null|      2000|
| Banana|      1000|       400|      2000|      null|
|Carrots|      1500|      1200|      2000|      null|
+-------+----------+----------+----------+----------+



dts: Seq[String] = ArrayBuffer(2021-07-01, 2021-07-02, 2021-07-03, 2021-07-04)
pivotDf: org.apache.spark.sql.DataFrame = [product: string, 2021-07-01: bigint ... 3 more fields]


In [135]:
val newColNames = (1 to dts.length+1).toList.map(x=>"before_"+x.toString)
var cols = col("product")+:dts.zip(newColNames).map(f=>{col(f._1).as(f._2)})
val newDf = pivotDf.select(cols:_*)
newDf.show()

+-------+--------+--------+--------+--------+
|product|before_1|before_2|before_3|before_4|
+-------+--------+--------+--------+--------+
| Orange|    4000|    4000|    null|    null|
|  Beans|    1600|    1500|    null|    2000|
| Banana|    1000|     400|    2000|    null|
|Carrots|    1500|    1200|    2000|    null|
+-------+--------+--------+--------+--------+



newColNames: List[String] = List(before_1, before_2, before_3, before_4, before_5)
cols: Seq[org.apache.spark.sql.Column] = ArrayBuffer(product, 2021-07-01 AS `before_1`, 2021-07-02 AS `before_2`, 2021-07-03 AS `before_3`, 2021-07-04 AS `before_4`)
newDf: org.apache.spark.sql.DataFrame = [product: string, before_1: bigint ... 3 more fields]
